# Introduction to Data Science 
# Lecture 28: Ratings, Rankings, and Elections
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll study methods for ratings, rankings, and elections. In particular, we'll discuss

* methods and applications for ratings and rankings 
* Arrow's impossibility theorem 
* rating/ranking in sports
* the least squares method for rating, a.k.a. HodgeRank 

Recommended Reading:
* A. N. Langville and C. D. Meyer, [Who's \#1?: The Science of Rating and Ranking](https://doi.org/10.1515/9781400841677), Princeton University Press (2012).
* C. Borgers, [Mathematics of Social Choice](https://doi.org/10.1137/1.9780898717624), SIAM (2010).


In [ ]:
# imports and setup
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

## Ratings vs. Rankings 

+ A *rating* assigns each item a numerical score. 
+ A *ranking* refers to a rank-ordered list of items. 

Rankings can be obtained from ratings by sorting. 

**Applications involving rating/ranking:** 
+ Sports (football, tennis, baseball, chess, ...) 
+ Ranking webpages (Google)
+ Online e-commerce (Amazon, Netflix, ...)
+ College Rankings (US News, ...)
+ Social networks (Facebook, ...)
+ Political elections 
+ ...



## Elections and Arrow's impossibility theorem 


### Elections vs. winner selection
We define an *election procedure* as a method for generating a ranking (ordering) of a set of candidates from voter preferences. Note that an election does not just name the top candidate, but instead, ranks all of the candidates. (This slightly differs from everyday usage.) We refer to a procedure that only decides the top candidate as a *winner selection method*. 

What is the relationship between these two problems? Clearly, one could recursively use a winner selection method   to define an election procedure. (First choose a winner, then remove the winner from the ballot and use the winner selection method to choose the runner-up, *etc*...) Conversely, the number one ranked alternative in an election procedure can be considered the winner, so every election procedure generates a winner selection method. So are these two problems---elections and winner selection---really equivalent? No, there are election procedures which cannot be derived from recursively selecting a winner.  Thus, election procedures are more general and winner selection methods should be considered a subset of election procedures. 

First, note that if there are only two candidates, then the  election and winner-selection methods are the same. In this case,  any reasonable method will select the candidate for which the *majority* of the electorate favors.  In the following discussion, we'll assume that there are three or more candidates. The interesting case arises when no single candidate receives a majority of the votes. 

**Examples:**  In the U. S., elections are generally decided by the plurality method or some variation thereof. (Presidential elections in the U.S. involve the electoral college, but I'm not going to discuss this here.) In the *plurality method*, each voter votes for a single candidate and the candidate with the most votes is declared the winner. It often occurs that the candidate does not receive a majority of the votes.  For example, George W. Bush was elected president in 2000 with 47.87% of the votes and Bill Clinton was elected president in 1992 with 43.01% of the votes. In both cases, "third-party" candidates (Ralph Nader and Ross Perot respectively) prevented the winner (or, possibly, the runner-up) from receiving a majority vote. Other voting methods may have resulted in alternative  outcomes in these two elections. For example, in France, Russia, and Brazil, a *runoff election* is held, in which only the top two candidates appear on the ballot. This idea can be further generalized to an election procedure called the *elimination method*, where the candidate with the smallest number of votes is removed from the ballot in each round. This method is used by the International Olympic Committee to select the host for the Olympic games. 


### Election methods
We write $a,b,c,\ldots$ to denote candidates and $i,j,k,\ldots$ to denote voters. 
Each voter has his/her own preferences. We write 
$a \succ_i b$
if candidate $a$ is preferred to candidate $b$ by voter $i$. If voter $i$ has no preference over candidates $a$ and $b$, we write $a=_i b$. Finally we write 
$a \succeq_i b$
to indicate that candidate $b$ is not preferred to candidate $a$ by voter $i$; that is, either: $a\succ_i b$ or $a=_i b$.  We require the voter preferences to satisfy the following relationships:
1. For each pair of candidates, $a$ and $b$,  exactly one of the following holds: $a\succ_i b$, $b\succ_i a$, or $a=_i b$. 
+ For all candidates $a$, $a=_i a$. 
+ Each voter ranking should give a transitive relation: $a \succeq_i b$ and $b \succeq_i c$ implies that $a \succeq_i c$ 
with $a=_i c$ if and only if $a=_i b$ and $b=_i c$. 

An *election method* is a method for generating a ranking of the candidates from the voter preferences that satisfy the above three properties. The relationships generated by the election procedure are  denoted by $\succ$, $=$, and $\succeq$. 

There are many different election methods, for example, the [Borda count](https://en.wikipedia.org/wiki/Borda_count) and [Copeland](https://en.wikipedia.org/wiki/Copeland%27s_method) methods. It can be shown that these methods have drastically different properties. 

**Example:** 
We say that a candidate is a *Condorcet candidate* if he beats every other candidate in head-to-head competition (that is, with all other candidates removed from the ballot).  It is not difficult to show that a Condorcet candidate does not necessarily exist, but if one does, it is unique. Intuitively it would seem desirable for a winner selection method to select the Condorcet candidate, if one exists. One can show that the Copeland  method has the property but the Borda count does not. 

This might prompt us to write down a list of desirable properties and then see which election methods satisfy which properties. 

![http://en.wikipedia.org/wiki/Voting_system](ElectionMethodProperties.png)

Each row of the table is an election method and each column is a desirable property. The entries of the table indicate whether or not the election method satisfies that property. Don't worry about how we define all of these methods or all of the properties. Note that Plurality, Runoff voting, Borda count, and Copeland are among the methods compared and  that the Condorcet property is the third column in the table. 


Looking at this table, we observe that none of the election methods satisfy all of the (desirable) properties. Of course, tomorrow someone could invent a new election method and we could then check to see which of these properties hold for that method.  This leads us to the following question: is it possible that there exists a method that  satisfies all (or possibly a chosen subset) of these properties?  

### Arrow's impossibility theorem
In the following, we define five sensible criteria which we expect a fair and reasonable  election procedure should satisfy. Surprisingly, in 1963, Kenneth Arrow proved that there does not exist an election procedure which can satisfy all five criteria. 

We say an election procedure is *fair* if it satisfies the following five criteria:
1. All conceivable voter rankings are allowed. 
2. If $a \succeq_i b$ for all voters $i$, then $a\succeq b$ with equality if and only if $a=_i b$ for all voters $i$. We interpret this to mean that unanimous opinions are respected. 
3. If in two different elections, each voter ranks candidates $a$ and $b$ the same, then the election outcomes between $a$ and $b$ are the same in the two elections. That is, how candidates are ranked relative to each other in an election depends only on how the voters rank them relative to each other and not how they are ranked relative to other candidates. Denoting preferences in the second election by the symbol $\supseteq$, we express this property in symbols: 
$$
 \text{If} \ \forall i, \ a\succeq_i b \ \iff \ a\supseteq_i b, \quad  \text{then} \ a\succeq b \iff a \supseteq b.  
$$
4. If there are two elections such that $a\succeq_i b$ implies $a \supseteq_i b$ for all $i$, and if also $a \succeq b$, then $a\supseteq b$. In other words, if $a$ does at least as well as $b$ in a later ranking by the voters as he did in the present ranking, and if he beat $b$ in the present election, he'll beat $b$ in the later election. 
5.  There is no voter $i$ such that $a\succ b$ if and only if $a\succ_i b$. In other words, there is no dictator. 

To illustrate the absurdity that the presence of the third option should not influence the voters preference between the other two, we consider the following anecdote attributed to Sidney Morgenbesser. 

```After finishing his meal, a diner decides to order dessert from a restaurant. The waitress tells him he has two choices: apple pie and blueberry pie. The diner orders the apple pie. After a few minutes the waitress returns and says that they also have cherry pie at which point the diner says: "In that case I'll have the blueberry pie." ```

**Arrow's Impossibility Theorem.** No election procedure for three or more candidates is fair. 

In practice, one has to be very careful when considering the method used for an election. No method is fair and, at worse, the method can be exploited by a coalition of the voters. 


## Rating/ranking in sports


*"At a Lawn Tennis Tournament where I chanced, some while ago, to be a spectator, the present method of assigning prizes was brought to my notice by the lamentations of one of the Players, who had been beaten (and had thus lost all chance of a prize) early in the contest, and who had had the mortification of seeing the 2nd prize carried off by a Player whom he knew to be quite inferior to himself."*
---Charles Dodgson a.k.a Lewis Carroll (1883) 


Below, it will be convenient to use the language of sports to describe rating/ranking methods (*e.g.*, items to be ranked are teams). But, keep in mind that the methods discussed are more general than sports applications. 

**Rating methods:**
+ Least squares methods (a.k.a. $\ell^2$ norm, HodgeRank, Massey, Colley, ...)
+ Least absolute deviation ($\ell^1$ norm)
+ Probabilistic methods such as the Bradley-Terry and Thurstone models
+ Methods based on the Perron-Frobenius theorem (Keener, PageRank,...)
+ Elo's method
+ ...


Here I've listed just a few methods for rating/ranking, but there are many more. The [masseyratings.com](http://www.masseyratings.com/cf/compare.htm) website has a comparison of 103 different rankings for NCAA college football. 

The simplest rating method rates teams based on their *winning percentage*. This is the method used in Major League Baseball (MLB). One drawback of this method is that it fails to take into account the strength of the teams' opponent (strength of schedule). 

We'll focus on the **least squares method**, which is perhaps the simplest method that does take into account the strength of opponents. 



## Least Squares Rating

**Data:** We start with a pairwise comparison dataset, consisting of 
+ a collection of $n$ items to be ranked,  $V =\{ i\}_{i=1}^n$
+ pairwise comparisons, $y_{i,j}$, comparing items $i$ and $j$
+ confidence weights, $w_{i,j}$, in the comparison between items $i$ and $j$.  

It might be the case that not all items in the dataset have been compared. For such pairs, $(i,j)$, we set the confidence weights $w_{i,j}=0$. We also require that the pairwise comparisons are anti-symmetric, *i.e.*, 
$y_{i,j} = - y_{j,i}$. This simply means that item $i$ is preferred to item $j$ as much as item $j$ is dispreferred to item $i$. 


**Goal:** Find a rating $\phi_i$ for $i\in V$ such that 
$$
\phi_i - \phi_j \approx y_{i,j}, 
$$
where the comparisons with high confidences are matched more closely than comparisons with a low confidence (as measured by $w_{i,j}$). 


One way to make this problem precise is to define the function 
$$
J(\phi) = \sum_{i,j} w_{i,j} (\phi_i - \phi_j - y_{i,j})^2,
$$ 
and then to minimize
$$
\min_{\phi\in \mathbb R^n} \ J(\phi). 
$$
A minimizer of this problem is  called the least squares rating or HodgeRank estimate. Note that in the objective function $J(\phi)$, we pay a penalty proportional to $w_{i,j}$ when $\phi_i - \phi_j \neq y_{i,j}$; the solution minimizes this cost. 


## Interpretation on a directed graph
We can interpret this problem on a directed graph as follows. A vertex is assigned to each item to be ranked. We then add directed edges (arcs) to the graph between each vertex pair for which there is a pairwise comparison.  The orientation of the arcs is chosen based on the signs of the pairwise preferences. If $y_{i,j}>0$, then our convention is choose the arc orientation $(i,j)$ instead of $(j,i)$. Thus, the equation $\phi_i - \phi_j \approx y_{i,j}$ is enforcing the requirement that item $i$ is preferred to item $j$. We enumerate the arcs $A = \{k\}_{k=1}^m$. The pairwise comparisons and confidence weights are then viewed as functions on the arc set, $A$. Let $D = (V,A)$. The *arc-vertex incidence matrix* for $D$ is $B \in \mathbb R^{m\times n}$, with entries  
$$
B_{k,j} = \begin{cases}
1 & j = \text{head}(k) \\
-1 & j = \text{tail}(k) \\
0 & \text{otherwise}. 
\end{cases}
$$
We can use the matrix $B$ to rewrite the function $J$, in a very compact form:
$$
J(\phi) = \| B \phi - y \|_{2,w}^2. 
$$
Here, $\| \cdot \|_{2,w} $ is the $w$-weighted $\ell^2$-norm, $\| v \|_{2,w} = \sqrt{ \sum_{k} w_k v_k^2} = \sqrt{v^t W v}$, where $W = \text{diag}(w)$.

Thus, the least squares rating is the solution to the optimization problem
$$
\min_{\phi\in \mathbb R^n} \  \| B \phi - y \|_{2,w}^2 
$$

**Solution.** 
How do you solve the least squares problem? We can take the derivative and set it to zero to obtain the "normal equations". We write 
$$
J(\phi) = \phi^t B^t W B \phi - 2 y^t W B \phi + y^t W y. 
$$
Taking the gradient with respect to $\phi$, we obtain 
$$
0 = \nabla_\phi J = 2 B^t W B \phi - 2 B^t W y  \qquad \implies \qquad  B^t W B \phi = B^t W y. 
$$
Thus, the minimizer is a solution to this linear system of equations. 

**Interpretation.** 
The matrix, $B^t W B$, on the left-hand-side of this equation has a nice interpretation. 
This matrix, $\Delta_w = B^t W B$,  is called the *$w$-weighted graph Laplacian*. The unweighted version, $\Delta \colon V \to V$, has entries 
$$
 \Delta_{i,j} = \begin{cases}
\text{deg}(i) &  i = j \\
-1 & i \sim j \\
0 & \text{otherwise}
\end{cases}
$$
We can also write $\Delta = D - A$ where $D$ is the degree matrix and $A$ is the adjacency matrix. 
The $w$-weighted graph Laplacian has entries 
$$
( \Delta_w)_{i,j} = \begin{cases}
\text{deg}_w(i) & i = j \\
-w_{i,j} & \text{otherwise} 
\end{cases}
$$
Here we have used the convention that $w_{i,j}=0$ if $i$ and $j$ are not adjacent and $\text{deg}_w(i) = \sum_{j \in V} w_{i,j}$ are the $w$-weighted degrees. We can also write $\Delta = D_w - W$ where $D_w$ is the $w$-weighted degree matrix and $W$ is the weight matrix. 



## Least Squares Rating: toy example

We consider the following hypothetical sports problem. There are four football teams: Utah (U), BYU (Y), Colorado (C), and Idaho (I). Five games are played with the following results:

U vs Y: 20 - 10 

Y vs C: 7 - 10

U vs C: 10 -10

U vs I: 10 - 7

I vs Y: 7 - 7

We first construct the pairwise comparisons, $y_{i,j}$ defined by
$$
y_{i,j} = \frac{\text{points team $j$ scored - points team $i$ scored}}{\text{total points in game}}. 
$$

In [ ]:
scores = np.array([(30, 10), (7, 10), (10, 10), (10, 7), (7, 7)])
print(scores)

y = (scores[:,1] - scores[:,0]) / (scores[:,0] + scores[:,1])
print(y)

We also number the teams.

In [ ]:
teams = ['Utah (U)','BYU (Y)','Colorado (C)','Idaho (I)']
for i,t in enumerate(teams):
    print(str(i) + ': ' + t)


Construct the arc-vertex incidence matrix
$$
B_{k,j} = \begin{cases}
1 & j = \textrm{head}(k) \\
-1 & j = \textrm{tail}(k) \\
0 & \textrm{otherwise}. 
\end{cases}
$$
This just keeps track of which teams have played which teams. 

In [ ]:
B = np.zeros((5, 4))

B[0,1] = 1; B[0,0] =-1;
B[1,2] = 1; B[1,1] =-1; 
B[2,2] = 1; B[2,0] =-1; 
B[3,3] = 1; B[3,0] =-1; 
B[4,1] = 1; B[4,3] =-1; 
print(B)

# now we have enough information just to print the  game results 
for i,sc in enumerate(y):
    head = np.where(B[i,:]==1)[0][0]
    tail = np.where(B[i,:]==-1)[0][0]
    print(teams[head] + ' vs. ' + teams[tail] + ': ' +str(sc))


We now use the *lstsq* function in the np.linalg library to find the least squares rating, solving the least squares problem, 
$$
\min_{\phi} \ \| B \phi - y \|^2. 
$$

In [ ]:
sol = np.linalg.lstsq(B,y,rcond=None)
phi = sol[0]
print(phi)

In [ ]:
ind = np.argsort(phi)
ind = np.flipud(ind)
sorted_phi = phi[ind]
sorted_teams = [teams[i] for i in ind]

for i,t in enumerate(sorted_teams):
    print(t + ': rating = ' + str(sorted_phi[i]))

## College Football Primer

National Collegiate Athletic Association (NCAA) College Football is divided into two subdivisions: 
- Football Bowl Subdivision (FBS), formerly  Division I-A
- Football Championship Subdivision (FCS), formerly Division I-AA

We'll focus on the FBS. 

The FBS is further divided into 11 conferences, some of which have sub-divisions. For example, the University of Utah is in the South Division of the Pacific 12 (Pac-12) Conference. 

      A.  American Athletic Conference
           i) East Division
                Central Florida
                Cincinnati
                Connecticut
                East Carolina
                South Florida
                Temple
          ii) West Division
                Houston
                Memphis
                Navy
                SMU
                Tulane
                Tulsa
      B.  Atlantic Coast Conference       
           i) Atlantic Division
                Boston College
                Clemson
                Florida St
                Louisville
                North Carolina St
                Syracuse
                Wake Forest
         ii) Coastal Division
                Duke
                Georgia Tech
                Miami FL
                North Carolina
                Pittsburgh
                Virginia
                Virginia Tech       
      C.  Big 10 Conference
           i) East Division
                Indiana
                Maryland
                Michigan
                Michigan St
                Ohio State
                Penn State
                Rutgers
         ii) West Division
                Illinois
                Iowa
                Minnesota
                Nebraska
                Northwestern
                Purdue
                Wisconsin      
      D.  Big 12 Conference
            Baylor
            Iowa St
            Kansas
            Kansas St
            Oklahoma
            Oklahoma St
            Texas
            TCU 
            Texas Tech
            West Virginia
      E.  Conference USA
           i) East Division
                Florida Atlantic
                Florida Int'l
                Marshall
                Middle Tennessee St
                UNC-Charlotte
                Old Dominion
                Western Kentucky
          ii) West Division
                Louisiana Tech
                North Texas
                Rice
                Southern Miss
                Texas-San Antonio
                UTEP                   
      F.  Mid-American Conference
           i) East Division
                Akron
                Bowling Green
                Buffalo
                Kent St
                Miami OH
                Ohio U.
          ii) West Division
                Ball St
                Central Michigan
                Eastern Michigan
                Northern Illinois
                Toledo
                Western Michigan
      G.  Mountain West Conference
           i) Mountain Division
                Air Force
                Boise St
                Colorado St
                New Mexico
                Utah St
                Wyoming
          ii) West Division
                Fresno St
                Hawai`i
                Nevada
                San Diego St
                San José St
                UNLV      
      H.  Pacific 12 Conference
           i) North Division
                California
                Oregon
                Oregon St
                Stanford
                Washington
                Washington St
          ii) South Division
                Arizona
                Arizona St
                Colorado
                Southern Cal
                UCLA  
                Utah        
      I.  Southeastern Conference
           i) Eastern Division
                Florida
                Georgia
                Kentucky
                Missouri
                South Carolina
                Tennessee
                Vanderbilt
          ii) Western Division
                Alabama
                Arkansas
                Auburn
                LSU
                Mississippi
                Mississippi St
                Texas A&M
      J.  Sun Belt Conference
            Appalachian St
            Arkansas St
            Georgia Southern
            Georgia St
            Idaho
            Louisiana-Lafayette
            Louisiana-Monroe
            New Mexico St
            South Alabama
            Texas St-San Marcos
            Troy
      K.  Division I FBS Independents
            Army
            Brigham Young
            Massachusetts
            Notre Dame  

More conference information available [here](http://prwolfe.bol.ucla.edu/cfootball/conferences.htm). 

## Download and clean data

We download the 2017 College Football game results from the 
[Massey Ratings website](http://masseyratings.com). 



### First download a list of college team names

In [ ]:
io = "teams.htm"
# io = "https://www.masseyratings.com/scores.php?s=295489&sub=295489&all=1&mode=3&format=2" # 2017
# io = "http://masseyratings.com/scores.php?s=286577&sub=286577&all=1&mode=3&format=2" # 2016
teams = pd.read_csv(io,names=['team'])
teams['team'] = teams['team'].apply(str.strip)
print("The number of teams is", teams.shape[0])
teams

In [ ]:
# where is Utah in the Pandas series teams? 
teams[teams['team']=='Utah']

In [ ]:
teams.loc[857]['team']

### Download the game results

In [ ]:
io = "scores.htm"
# io = "https://www.masseyratings.com/scores.php?s=295489&sub=295489&all=1&mode=3&format=1" #2017
# io = "http://masseyratings.com/scores.php?s=286577&sub=286577&all=1&mode=3&format=1" #2016
df = pd.read_csv(io,names=['id','date','team1','homefield1','score1','team2','homefield2','score2'])
print("The number of games is", df.shape[0])
df

### Clean the data
We'll ignore the homefield and add columns with the team's name, rather than just the team id. 

In [ ]:
# add a new column with team names
df.drop(['homefield1','homefield2'],inplace=True,axis=1)
df.insert(3, 'team_name1', df['team1'].map(lambda i: teams['team'][i]))
df.insert(6, 'team_name2', df['team2'].map(lambda i: teams['team'][i]))
df

## Consider only Pac 12 teams

Our goal will be to rank the Pac 12 teams based on intra-conference games. 

In [ ]:
P12 = ['California', 'Oregon', 'Oregon_St', 'Stanford', 'Washington', 'Washington_St', 
    'Arizona', 'Arizona_St', 'Colorado', 'USC', 'UCLA', 'Utah'] 
num_P12_teams = len(P12)

# get PAC12 teams from teams
P12_ind = teams[teams['team'].isin(P12)].index.tolist()  
P12_teams = teams.loc[P12_ind]

# assign a new ordering for teams
P12_teams['P12_ind'] = np.arange(num_P12_teams)
P12_teams['global_ind'] = P12_teams.index
P12_teams.set_index('P12_ind',inplace=True)

P12_teams

In [ ]:
P12_df = df[(df['team_name1'].isin(P12)) & (df['team_name2'].isin(P12))].copy()
num_P12_games = P12_df.shape[0]
print(num_P12_games)
P12_df

## Use the Least Squares method to construct a rating

We first construct the pairwise comparisons, $y_{i,j}$ defined by
$$
y_{i,j} = \frac{\text{points team $j$ scored - points team $i$ scored}}{\text{total points in game}}. 
$$


In [ ]:
P12_df['y'] = (P12_df['score1'] - P12_df['score2']) / (P12_df['score1'] + P12_df['score2'])
y = P12_df['y'].tolist()
P12_df

Construct the arc-vertex incidence matrix
$$
B_{k,j} = \begin{cases}
1 & j = \textrm{head}(k) \\
-1 & j = \textrm{tail}(k) \\
0 & \textrm{otherwise}. 
\end{cases}
$$
This just keeps track of which teams played in each game. 

In [ ]:
# first we need to reorder the teams in the PAC12 ordering

print(P12_teams['global_ind'].tolist())

glob_P12_dict = {j:i for i,j in enumerate(P12_teams['global_ind'].tolist())}
print(glob_P12_dict)

In [ ]:
# construct B

B = np.zeros((num_P12_games, num_P12_teams))

for ii,g in enumerate(P12_df.index):
    team1_global_ind = P12_df['team1'][g]
    team1_P12_ind = glob_P12_dict[team1_global_ind]    
    B[ii,team1_P12_ind] = 1

    team2_global_ind = P12_df['team2'][g]
    team2_P12_ind = glob_P12_dict[team2_global_ind]    
    B[ii,team2_P12_ind] = -1


In [ ]:
# now we have enough information just to print the  game results 
for i,sc in enumerate(y):
    head = np.where(B[i,:]==1)[0][0]
    tail = np.where(B[i,:]==-1)[0][0]
    print(P12_teams['team'][head] + ' vs. ' + P12_teams['team'][tail] + ': ' +str(sc))

We now use the *lstsq* function in the np.linalg library to find the least squares rating, solving the least squares problem, 
$$
\min_{\phi} \ \| B \phi - y \|^2. 
$$

In [ ]:
phi = np.linalg.lstsq(B,y,rcond=.1)[0]
print(phi)

In [ ]:
P12_teams['rating'] = phi
print(P12_teams)

## Sort the ratings to generate a ranking


In [ ]:
P12_rankings = P12_teams.sort_values('rating', axis=0, ascending=False)
P12_rankings['ranking'] = np.arange(1,num_P12_teams+1)
P12_rankings.set_index('ranking',inplace=True)
P12_rankings.drop('global_ind',axis=1,inplace=True)
print(P12_rankings)

Compare against the 2017 PAC 12 rankings [here](https://en.wikipedia.org/wiki/2017_Pac-12_Conference_football_season).

## Visualize the schedule for the Pac-12 teams

In [ ]:
# make graph
Lap = np.dot(np.transpose(B),B)
adj = -Lap + np.diag(np.diag(Lap))
game_graph = nx.from_numpy_array(adj)

# Calculate the layout positions first
pos = nx.spring_layout(game_graph)

# labeling needs a dictionary
label_dict = {i:j for i,j in enumerate(P12_teams['team'].tolist())}

# draw graph
nx.draw_networkx(game_graph, pos=pos, node_size=3000, labels = label_dict, node_shape='s')
plt.show()


The above network representation may distort what's going on. Let's try another layout and also try some additional encodings.

In [ ]:
# Calculate size from rankings
P12_sizes = [4000 * (x + 1) for x in P12_teams['rating']]
P12_colors = [((0.4 - x), (0.6 - x), 1.0) for x in P12_teams['rating']]

# Rather than calculating a circular layout and then applying, we can ask networkx to do so directly
nx.draw_circular(game_graph, node_size=P12_sizes, node_color=P12_colors, labels = label_dict)
plt.show()

**Exercises:** 
1. Use the least squares method to rank all of the teams (not just the Pac-12 ones).
+ Use the least squares method to find the 2018 rankings. 